In [ ]:
from google.colab import drive
import pandas as pd
from tqdm import tqdm

drive.mount('/content/drive')
path = '/content/drive/MyDrive/6000_IMDB_Dataset.csv'

Mounted at /content/drive


In [ ]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from transformers import AdamW
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

review_df = pd.read_csv(path)
feature = review_df['review'].tolist()
label = review_df['sentiment'].tolist()

for i in range(len(label)):
    if label[i] == 'positive':
        label[i] = 1
    else :
        label[i] = 0

train_texts,test_texts, train_labels,test_labels = train_test_split(feature, label, train_size=0.7, shuffle=True)

# 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')

# 모델 준비
class BERT(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_attention_heads, num_hidden_layers):
        super(BERT, self).__init__()
        self.embeddings = BERTEmbeddings(vocab_size, embed_size)
        self.encoder = BERTEncoder(hidden_size, num_attention_heads, num_hidden_layers)

    def forward(self, input_ids, attention_mask):
        embedding_output = self.embeddings(input_ids)
        encoder_output = self.encoder(embedding_output, attention_mask)
        return encoder_output

class BERTEmbeddings(nn.Module):
    def __init__(self, vocab_size, embed_size):
        super(BERTEmbeddings, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embed_size)
        self.position_embeddings = nn.Embedding(512, embed_size)
        self.LayerNorm = nn.LayerNorm(embed_size, eps=1e-12)
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids):
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        word_embeddings = self.word_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        embeddings = word_embeddings + position_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

class BERTEncoder(nn.Module):
    def __init__(self, hidden_size, num_attention_heads, num_hidden_layers):
        super(BERTEncoder, self).__init__()
        layer = BERTLayer(hidden_size, num_attention_heads)
        self.layers = nn.ModuleList([layer for _ in range(num_hidden_layers)])

    def forward(self, hidden_states, attention_mask):
        for layer_module in self.layers:
            hidden_states = layer_module(hidden_states, attention_mask)
        return hidden_states

class BERTLayer(nn.Module):
    def __init__(self, hidden_size, num_attention_heads):
        super(BERTLayer, self).__init__()
        self.attention = MultiHeadAttention(hidden_size, num_attention_heads)
        self.intermediate = nn.Linear(hidden_size, hidden_size)
        self.output = nn.Linear(hidden_size, hidden_size)
        self.LayerNorm = nn.LayerNorm(hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(0.1)

    def forward(self, hidden_states, attention_mask):
        attention_output = self.attention(hidden_states, attention_mask)
        intermediate_output = F.relu(self.intermediate(attention_output))
        layer_output = self.output(intermediate_output)
        layer_output = self.LayerNorm(layer_output + attention_output)
        layer_output = self.dropout(layer_output)
        return layer_output

class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_size, num_attention_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_attention_heads = num_attention_heads
        self.hidden_size = hidden_size
        self.attention_head_size = hidden_size // num_attention_heads
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(hidden_size, self.all_head_size)
        self.key = nn.Linear(hidden_size, self.all_head_size)
        self.value = nn.Linear(hidden_size, self.all_head_size)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(hidden_size, hidden_size)

    def forward(self, hidden_states, attention_mask):
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)

        query_layer = self._split_heads(mixed_query_layer)
        key_layer = self._split_heads(mixed_key_layer)
        value_layer = self._split_heads(mixed_value_layer)

        batch_size, num_heads, seq_length, _ = query_layer.size()
        attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
        attention_mask = attention_mask.repeat(1, num_heads, 1, 1)

        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2)) / math.sqrt(self.attention_head_size)
        attention_scores = attention_scores + attention_mask
        attention_probs = nn.Softmax(dim=-1)(attention_scores)
        attention_probs = self.dropout(attention_probs)

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = self._combine_heads(context_layer)
        context_layer = self.out_proj(context_layer)
        return context_layer

    def _split_heads(self, x):
        batch_size, seq_length, hidden_size = x.size()
        x = x.view(batch_size, seq_length, self.num_attention_heads, self.attention_head_size)
        return x.permute(0, 2, 1, 3)

    def _combine_heads(self, x):
        batch_size, num_attention_heads, seq_length, _ = x.size()
        x = x.permute(0, 2, 1, 3).contiguous()
        return x.view(batch_size, seq_length, self.hidden_size)

class BERTClassifier(nn.Module):
    def __init__(self, bert, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(bert.encoder.layers[-1].output.in_features, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask)
        pooled_output = outputs[:, 0]  # [CLS] 토큰의 출력을 사용
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits.view(-1, self.classifier.out_features), labels.view(-1))

        return logits, loss


In [ ]:
vocab_size = 30522
embed_size = 768
hidden_size = 768
num_classes = 2
num_attention_heads = 12
num_hidden_layers = 12
#bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model = BERT(vocab_size, embed_size, hidden_size, num_attention_heads, num_hidden_layers)
model = BERTClassifier(bert_model, num_classes)

# 훈련 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 데이터셋 생성
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

epoch_num = 1
model.train()
for epoch in range(epoch_num):
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        print()
        print(outputs)
        print(labels)
        print(torch.argmax(outputs, dim=1))
        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

# 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs
        predicted_labels = torch.argmax(logits, dim=1)
        total += labels.size(0)
        correct += (predicted_labels == labels).sum().item()


accuracy = correct / total
print(f'테스트 정확도: {accuracy}')

  0%|          | 0/525 [00:00<?, ?it/s]


tensor([[ 0.6337,  0.5983],
        [ 0.7738,  0.2087],
        [ 0.3197,  0.3420],
        [ 0.8605,  0.2279],
        [ 0.0373,  0.3898],
        [ 0.7157,  1.1144],
        [ 0.6702,  0.8480],
        [-0.0496, -0.2568]], grad_fn=<AddmmBackward0>)
tensor([1, 0, 1, 1, 1, 0, 0, 1])
tensor([0, 0, 1, 0, 1, 1, 1, 0])


  0%|          | 1/525 [00:02<24:53,  2.85s/it]


tensor([[-1.4905,  2.0268],
        [-0.8479,  1.1084],
        [-0.9988,  2.4992],
        [-1.2539,  2.5697],
        [-1.4904,  2.1142],
        [-1.6487,  2.5770],
        [-1.2388,  2.3740],
        [-1.6296,  2.1385]], grad_fn=<AddmmBackward0>)
tensor([1, 1, 0, 1, 0, 1, 0, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1])


  0%|          | 2/525 [00:05<25:02,  2.87s/it]

...


tensor([[ 0.1186,  0.3710],
        [ 0.3879,  0.7756],
        [ 0.2928,  0.4677],
        [ 0.1285,  0.1055],
        [ 0.3577,  0.3073],
        [ 0.1958,  0.2165],
        [ 0.4930,  0.3152],
        [-0.0504,  0.2142]], grad_fn=<AddmmBackward0>)
tensor([0, 0, 0, 0, 1, 0, 0, 0])
tensor([1, 1, 1, 0, 0, 1, 0, 1])


100%|█████████▉| 524/525 [24:33<00:02,  2.76s/it]


tensor([[ 0.0777,  0.1869],
        [ 0.3956,  0.1043],
        [ 0.2616,  0.6662],
        [ 0.3345,  0.5965],
        [ 0.3702,  0.3539],
        [-0.2988,  0.1576],
        [-0.1989,  0.7025],
        [-0.1861,  0.1231]], grad_fn=<AddmmBackward0>)
tensor([1, 0, 1, 1, 0, 0, 1, 0])
tensor([1, 0, 1, 1, 0, 1, 1, 1])


100%|██████████| 525/525 [24:35<00:00,  2.81s/it]


테스트 정확도: 0.5094444444444445
